In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

!unzip -q dogs-vs-cats.zip -d /content/cats-vs-dogs

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:05<00:00, 216MB/s]
100% 1.06G/1.06G [00:05<00:00, 219MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision import models


In [ ]:
# Define transformations for data augmentation and normalization
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(40),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

transform_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
# Paths for dataset
train_dir = '/content/cats-vs-dogs/train'
test_dir = '/content/cats-vs-dogs/test'

In [ ]:
# Load dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_test)


In [ ]:
# DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 32 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

In [ ]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training loop
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)
            labels = labels.unsqueeze(1)  # Reshape labels for BCE loss
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%')

# Train model
train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch [1/10], Loss: 0.6598, Accuracy: 59.94%
Epoch [2/10], Loss: 0.6025, Accuracy: 68.25%
Epoch [3/10], Loss: 0.5556, Accuracy: 71.83%
Epoch [4/10], Loss: 0.5261, Accuracy: 74.40%
Epoch [5/10], Loss: 0.4970, Accuracy: 76.33%
Epoch [6/10], Loss: 0.4749, Accuracy: 77.58%
Epoch [7/10], Loss: 0.4561, Accuracy: 78.71%
Epoch [8/10], Loss: 0.4398, Accuracy: 79.97%
Epoch [9/10], Loss: 0.4250, Accuracy: 81.16%
Epoch [10/10], Loss: 0.4132, Accuracy: 81.82%


### Transfer learning


In [ ]:
# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.features.parameters():
    param.requires_grad = False

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 74.9MB/s]


In [ ]:
vgg16.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 256),  # Fix input size to 512*7*7 = 25088
    nn.ReLU(),
    nn.Linear(256, 1),
    nn.Sigmoid()
)


In [ ]:
vgg16 = vgg16.to(device)

In [ ]:
# Compile new model
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)

# Train the VGG16 model
train_model(vgg16, train_loader, criterion, optimizer, epochs=1)


Epoch [1/1], Loss: 0.1144, Accuracy: 95.79%
